In [ ]:
#Triangles plotting lines pxl up from endpoint

In [ ]:
#!/usr/bin/env python3
# Ultra-fast, batched pipeline with threaded I/O + per-frame timing & counts
# Now renders ALL masks + triangles coloured by the mask sampled N px above the tip
# + Scout lines: per-pixel coloured line from tip to sample point (render-only)

import os, glob, sys, time
import cv2, torch, numpy as np
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from ultralytics import YOLO

# =======================
# Config
# =======================
home       = os.path.expanduser("~")
weights    = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
frames_dir = Path(home) / "Documents" / "GitHub" / "Ai-plays-SubwaySurfers" / "frames"
out_dir    = Path(home) / "Documents" / "GitHub" / "Ai-plays-SubwaySurfers" / "out_overlays2"
out_dir.mkdir(parents=True, exist_ok=True)

RAIL_ID    = 9
IMG_SIZE   = 512
CONF, IOU  = 0.30, 0.45
MAX_DET    = 30

# Color/region filter
TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

# Heat/triangle
HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
MIN_DARK_FRACTION   = 0.15
TRI_SIZE_PX         = 18

# Triangle mask scan distance (N pixels above tip)
SAMPLE_UP_PX        = 65

# Colours (BGR)
COLOR_GREEN  = (0, 255, 0)
COLOR_PINK   = (203, 192, 255)  # readable pink
COLOR_YELLOW = (0, 255, 255)
COLOR_RED    = (0, 0, 255)

# Runtime
BATCH               = 1
THREADS_IO          = max(2, (os.cpu_count() or 4) // 2)
SHOW_FIRST_N        = None   # None → all frames
RENDER_FIRST_N      = 150     # render overlays for first N frames only

# =======================
# System/Backends
# =======================
cv2.setUseOptimized(True)
try: cv2.setNumThreads(max(1, (os.cpu_count() or 1) - 1))
except Exception: pass

if torch.cuda.is_available():
    device, half = 0, True
    torch.backends.cudnn.benchmark = True
    try: torch.set_float32_matmul_precision('high')
    except Exception: pass
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model
# =======================
model = YOLO(weights)
try: model.fuse()
except Exception: pass

# Warmup
_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(_dummy, task="segment", imgsz=IMG_SIZE, device=device,
                  conf=CONF, iou=IOU, verbose=False, half=half, max_det=MAX_DET)

# =======================
# Precomputed
# =======================
TARGETS_BGR_F32 = np.array([(r,g,b)[::-1] for (r,g,b) in TARGET_COLORS_RGB], dtype=np.float32)
TOL2            = TOLERANCE * TOLERANCE

CLASS_COLOURS = {
    0:(255,255,0),1:(192,192,192),2:(0,128,255),3:(0,255,0),
    4:(255,0,255),5:(0,255,255),6:(255,128,0),7:(128,0,255),
    8:(0,0,128),9:(0,0,255),10:(128,128,0),11:(255,255,102)
}
LABELS = {
    0:"BOOTS",1:"GREYTRAIN",2:"HIGHBARRIER1",3:"JUMP",4:"LOWBARRIER1",
    5:"LOWBARRIER2",6:"ORANGETRAIN",7:"PILLAR",8:"RAMP",9:"RAILS",
    10:"SIDEWALK",11:"YELLOWTRAIN"
}

SAFE_GREEN = {9, 10}          # rails or sidewalk or no mask -> green
WARN_YELLOW = {2,3,4,5,8}   # barrier/jump/train/ramp -> yellow

# =======================
# Helpers
# =======================
def load_image_with_time(path: str):
    t0 = time.perf_counter()
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    t1 = time.perf_counter()
    return img, (t1 - t0) * 1000.0

def chunked(iterable, n):
    for i in range(0, len(iterable), n):
        yield iterable[i:i+n]

def highlight_rails_mask_only_fast(img_bgr, rail_mask):
    H, W = img_bgr.shape[:2]
    if not rail_mask.any():
        return np.zeros((H, W), dtype=bool)

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max()+1
    x0, x1 = xs.min(), xs.max()+1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - TARGETS_BGR_F32[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    colour_hit = np.any(dist2 <= TOL2, axis=-1)

    combined = np.logical_and(colour_hit, mask_roi)
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)
    if n <= 1:
        return np.zeros((H, W), dtype=bool)

    good = np.zeros_like(combined)
    areas  = stats[1:, cv2.CC_STAT_AREA]
    hs     = stats[1:, cv2.CC_STAT_HEIGHT]
    keep   = np.where((areas >= MIN_REGION_SIZE) & (hs >= MIN_REGION_HEIGHT))[0] + 1
    for k in keep:
        good[lbls == k] = True

    full = np.zeros((H, W), dtype=bool)
    full[y0:y1, x0:x1] = good
    return full

def red_vs_green_score(red_mask, green_mask):
    k = (HEAT_BLUR_KSIZE, HEAT_BLUR_KSIZE)
    r = cv2.blur(red_mask.astype(np.float32), k)
    g = cv2.blur(green_mask.astype(np.float32), k)
    diff = r - g
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2.0 * amax) + 0.5)
    return np.clip(norm * 255.0, 0, 255.0).astype(np.uint8)

def purple_triangles(score, H):
    top_ex = int(H * EXCLUDE_TOP_FRAC)
    bot_ex = int(H * EXCLUDE_BOTTOM_FRAC)

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    if top_ex: dark[:top_ex, :] = 0
    if bot_ex: dark[-bot_ex:, :] = 0

    dark = cv2.morphologyEx(
        dark, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9)),
        iterations=1
    )
    total_dark = int(dark.sum())
    if total_dark == 0:
        return [], None

    frac_thresh = int(np.ceil(MIN_DARK_FRACTION * total_dark))
    n_lbl, lbls, stats, _ = cv2.connectedComponentsWithStats(dark, 8)
    if n_lbl <= 1:
        return [], None

    tris = []
    for lbl in range(1, n_lbl):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area >= MIN_DARK_RED_AREA and area >= frac_thresh:
            ys, xs = np.where(lbls == lbl)
            if ys.size == 0:
                continue
            y_top = ys.min()
            x_mid = int(xs[ys == y_top].mean())
            tris.append((int(x_mid), int(y_top)))

    if not tris:
        return [], None

    best = min(tris, key=lambda xy: xy[1])
    return tris, best

# ---- Triangle classification by sampling masks N px above tip (no resizes) ----
def classify_triangles_at_sample(tri_positions, masks_np, classes_np, frame_H, frame_W, sample_up=SAMPLE_UP_PX):
    """
    For each triangle (x,y), sample (x, y - N) and determine which mask/class covers it.
    If no mask there: GREEN. If class==0: PINK. If class in {2,3,4,5,6,8}: YELLOW.
    If class in {9,10}: GREEN. Else: RED.
    Uses scale mapping into masks grid; avoids resizing masks.
    """
    if masks_np is None or classes_np is None or len(tri_positions) == 0:
        return []

    mh, mw = masks_np.shape[1], masks_np.shape[2]
    sx = (mw - 1) / max(1, (frame_W - 1))
    sy = (mh - 1) / max(1, (frame_H - 1))

    colours = []
    for (x, y) in tri_positions:
        ys = max(0, y - sample_up)
        mx = int(round(x * sx))
        my = int(round(ys * sy))
        if mx < 0: mx = 0
        elif mx >= mw: mx = mw - 1
        if my < 0: my = 0
        elif my >= mh: my = mh - 1

        cls_here = None
        for m, c in zip(masks_np, classes_np):
            if m[my, mx] > 0.5:  # mask hit
                cls_here = int(c)
                break

        if (cls_here is None) or (cls_here in SAFE_GREEN):
            colours.append(COLOR_GREEN)
        elif cls_here == 0:
            colours.append(COLOR_PINK)
        elif cls_here in WARN_YELLOW:
            colours.append(COLOR_YELLOW)
        else:
            colours.append(COLOR_RED)

    return colours

# --- NEW (render-only): colour for any frame-point via masks (no resizes) ---
def _colour_for_point(x, y, masks_np, classes_np, frame_H, frame_W):
    """
    Determine display colour at frame point (x,y) using same mapping rules as classifier.
    No mask or class in SAFE_GREEN -> GREEN; class==0 -> PINK; class in WARN_YELLOW -> YELLOW; else RED.
    """
    if masks_np is None or classes_np is None or masks_np.size == 0:
        return COLOR_GREEN
    mh, mw = masks_np.shape[1], masks_np.shape[2]
    # Precompute scales
    sx = (mw - 1) / max(1, (frame_W - 1))
    sy = (mh - 1) / max(1, (frame_H - 1))
    mx = int(round(x * sx))
    my = int(round(y * sy))
    if mx < 0: mx = 0
    elif mx >= mw: mx = mw - 1
    if my < 0: my = 0
    elif my >= mh: my = mh - 1

    cls_here = None
    for m, c in zip(masks_np, classes_np):
        if m[my, mx] > 0.5:
            cls_here = int(c)
            break

    if (cls_here is None) or (cls_here in SAFE_GREEN):
        return COLOR_GREEN
    if cls_here == 0:
        return COLOR_PINK
    if cls_here in WARN_YELLOW:
        return COLOR_YELLOW
    return COLOR_RED

# Returns: (tri_best_xy, tri_count, mask_count, to_cpu_ms, post_ms, masks_np, classes_np, rail_mask, green, tri_positions, tri_colours)
def process_frame_post(frame_bgr, yolo_res):
    H, W = frame_bgr.shape[:2]
    if yolo_res.masks is None:
        return None, 0, 0, 0.0, 0.0, None, None, None, None, [], []

    t0_to_cpu = time.perf_counter()
    masks_np = yolo_res.masks.data.cpu().numpy()  # [n,h,w]
    mask_count = int(masks_np.shape[0])
    if hasattr(yolo_res.masks, "cls") and yolo_res.masks.cls is not None:
        classes_np = yolo_res.masks.cls.cpu().numpy().astype(int)
    else:
        classes_np = yolo_res.boxes.cls.cpu().numpy().astype(int)
    t1_to_cpu = time.perf_counter()
    to_cpu_ms = (t1_to_cpu - t0_to_cpu) * 1000.0

    if mask_count == 0 or classes_np.size == 0:
        return None, 0, mask_count, to_cpu_ms, 0.0, masks_np, classes_np, None, None, [], []

    rail_sel = (classes_np == RAIL_ID)
    if not np.any(rail_sel):
        return None, 0, mask_count, to_cpu_ms, 0.0, masks_np, classes_np, None, None, [], []

    t0_post = time.perf_counter()
    rail_masks = masks_np[rail_sel].astype(bool)        # [k,h,w]
    union = np.any(rail_masks, axis=0).astype(np.uint8) # [h,w]
    rail_mask = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST).astype(bool)

    green = highlight_rails_mask_only_fast(frame_bgr, rail_mask)
    red   = np.logical_and(rail_mask, np.logical_not(green))
    score = red_vs_green_score(red, green)
    tri_positions, tri_best = purple_triangles(score, H)

    # classify triangles by sampling masks above tip
    tri_colours = classify_triangles_at_sample(tri_positions, masks_np, classes_np, H, W, SAMPLE_UP_PX)

    t1_post = time.perf_counter()
    post_ms = (t1_post - t0_post) * 1000.0

    return tri_best, len(tri_positions), mask_count, to_cpu_ms, post_ms, masks_np, classes_np, rail_mask, green, tri_positions, tri_colours

# --- rendering (excluded from timing) ---
def draw_triangle(img, x, y, size=TRI_SIZE_PX, colour=COLOR_RED):
    h = int(size * 1.2)
    pts = np.array([[x, y], [x-size, y+h], [x+size, y+h]], np.int32)
    cv2.fillConvexPoly(img, pts, colour)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, (0,0,0), 1, cv2.LINE_AA)

def render_overlays(frame_bgr, masks_np, classes_np, rail_mask, green_mask, tri_positions, tri_colours):
    """Draw all masks (class color) + labels, rail tint/green, coloured triangles, and scout lines on a copy of original frame."""
    out = frame_bgr.copy()
    H, W = out.shape[:2]
    alpha = 0.45

    if masks_np is not None and classes_np is not None and masks_np.size:
        for m, c in zip(masks_np, classes_np):
            m_full = m
            if m.shape != (H, W):
                m_full = cv2.resize(m.astype(np.uint8), (W, H), interpolation=cv2.INTER_NEAREST).astype(bool)
            color = CLASS_COLOURS.get(int(c), (255,255,255))
            out[m_full] = (np.array(color, dtype=np.uint8) * alpha + out[m_full] * (1 - alpha)).astype(np.uint8)
            ys, xs = np.where(m_full)
            if xs.size:
                xc, yc = int(xs.mean()), int(ys.mean())
                label = LABELS.get(int(c), f"C{int(c)}")
                cv2.putText(out, label, (max(5, xc-40), max(20, yc)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 2, cv2.LINE_AA)
                cv2.putText(out, label, (max(5, xc-40), max(20, yc)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)

    if rail_mask is not None:
        tint = out.copy()
        tint[rail_mask] = (0, 0, 255)  # red tint for rails
        out = cv2.addWeighted(tint, 0.30, out, 0.70, 0)
    if green_mask is not None:
        out[green_mask] = (0, 255, 0)

    # --- Scout lines (render-only): per-pixel coloured vertical line up to sample point ---
    if tri_positions:
        for (x, y) in tri_positions:
            y_end = max(0, y - SAMPLE_UP_PX)
            # Draw 1px dots along path; color per mask under that pixel
            # (Cheap: ≤ SAMPLE_UP_PX points per triangle)
            for yy in range(y, y_end - 1, -1):
                col = _colour_for_point(x, yy, masks_np, classes_np, H, W)
                # draw a 1px point
                out[yy, x] = col  # direct assign faster than cv2.circle for 1px

    # Draw triangles with per-triangle colours (after scout lines so tips are visible)
    for (x, y), col in zip(tri_positions, tri_colours):
        draw_triangle(out, x, y, colour=col)

    return out

# =======================
# Batched execution with prints; overlays saved for first N
# =======================
def run_pipeline_with_prints_and_overlays():
    paths = (
        glob.glob(str(frames_dir/"frame_*.jpg")) +
        glob.glob(str(frames_dir/"frame_*.png")) +
        glob.glob(str(frames_dir/"*.jpg")) +
        glob.glob(str(frames_dir/"*.png"))
    )
    paths = sorted(set(paths))
    if not paths:
        raise FileNotFoundError(f"No images in: {frames_dir}")
    if SHOW_FIRST_N is not None:
        paths = paths[:SHOW_FIRST_N]

    N = len(paths)
    results_triangle_xy = [None] * N

    def load_batch(batch_paths):
        imgs = [None] * len(batch_paths)
        read_ms = [0.0] * len(batch_paths)
        with ThreadPoolExecutor(max_workers=THREADS_IO) as ex:
            fut2idx = {ex.submit(load_image_with_time, p): i for i, p in enumerate(batch_paths)}
            for fut in as_completed(fut2idx):
                i = fut2idx[fut]
                img, r_ms = fut.result()
                imgs[i] = img
                read_ms[i] = r_ms
        ok = [(p, im, rm) for p, im, rm in zip(batch_paths, imgs, read_ms) if im is not None]
        if not ok:
            return [], [], []
        b_paths, b_imgs, b_read = zip(*ok)
        return list(b_paths), list(b_imgs), list(b_read)

    idx_global = 0
    for batch_paths in chunked(paths, BATCH):
        batch_paths, imgs_bgr, read_ms_list = load_batch(batch_paths)
        B = len(imgs_bgr)
        if B == 0:
            idx_global += len(batch_paths)
            continue

        t0_inf = time.perf_counter()
        res_list = model.predict(
            imgs_bgr, task="segment", imgsz=IMG_SIZE, device=device,
            conf=CONF, iou=IOU, verbose=False, half=half, max_det=MAX_DET,
            batch=B
        )
        try:
            if device == 0 and torch.cuda.is_available():
                torch.cuda.synchronize()
            elif device == "mps" and getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
                torch.mps.synchronize()
        except Exception:
            pass
        t1_inf = time.perf_counter()
        infer_ms_share = ((t1_inf - t0_inf) * 1000.0) / B

        for j, (img, yres, read_ms) in enumerate(zip(imgs_bgr, res_list, read_ms_list)):
            (tri_best_xy, tri_count, mask_count, to_cpu_ms, post_ms,
             masks_np, classes_np, rail_mask, green_mask, tri_positions, tri_colours) = process_frame_post(img, yres)

            results_triangle_xy[idx_global + j] = tri_best_xy
            proc_ms = infer_ms_share + to_cpu_ms + post_ms
            fname = os.path.basename(batch_paths[j])
            frame_idx = idx_global + j + 1

            print(f"[{frame_idx}/{N}] {fname}  "
                  f"read {read_ms:.1f} | infer {infer_ms_share:.1f} | "
                  f"to_cpu {to_cpu_ms:.1f} | post {post_ms:.1f} | "
                  f"masks {mask_count} | triangles {tri_count} "
                  f"=> proc {proc_ms:.1f} ms")

            # --- RENDERING (EXCLUDED from timing) ---
            if frame_idx <= RENDER_FIRST_N:
                overlay = render_overlays(img, masks_np, classes_np, rail_mask, green_mask, tri_positions, tri_colours)
                out_path = out_dir / f"overlay_{frame_idx:04d}_{fname}"
                cv2.imwrite(str(out_path), overlay)

        idx_global += B

    return results_triangle_xy

# =======================
# Entry
# =======================
if __name__ == "__main__":
    _ = run_pipeline_with_prints_and_overlays()


In [ ]:
#Jakes triangle finder final - Optimal saved function runs iwth jake lane + overlay saving

In [3]:
#!/usr/bin/env python3
# Ultra-fast, batched pipeline with threaded I/O + per-frame timing & counts
# Renders ALL masks + triangles coloured by the mask sampled N px above the tip
# + Scout lines (render only)
# + Starburst lines from each triangle tip to Jake's lane point
# + NEW: Lane-bearing selector: choose triangle whose signed degrees-from-vertical
#       is closest to the lane's target (left=+10.7°, mid=+1.5°, right=-15.0°)

import os, glob, sys, time, math
import cv2, torch, numpy as np
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from ultralytics import YOLO

# =======================
# Config
# =======================
home       = os.path.expanduser("~")
weights    = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
frames_dir = Path(home) / "Documents" / "GitHub" / "Ai-plays-SubwaySurfers" / "frames"

# SAVE HERE
out_dir    = Path(home) / "Documents" / "GitHub" / "Ai-plays-SubwaySurfers" / "out_overlays4"
out_dir.mkdir(parents=True, exist_ok=True)

RAIL_ID    = 9
IMG_SIZE   = 512
CONF, IOU  = 0.30, 0.45
MAX_DET    = 30

# Color/region filter
TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

# Heat/triangle
HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
MIN_DARK_FRACTION   = 0.15
TRI_SIZE_PX         = 18

# Triangle mask scan distance (N pixels above tip)
SAMPLE_UP_PX        = 65

# Colours (BGR)
COLOR_GREEN  = (0, 255, 0)
COLOR_PINK   = (203, 192, 255)
COLOR_YELLOW = (0, 255, 255)
COLOR_RED    = (0, 0, 255)
COLOR_WHITE  = (255, 255, 255)
COLOR_CYAN   = (255, 255, 0)
COLOR_BLACK  = (0, 0, 0)

# Runtime
BATCH               = 1
THREADS_IO          = max(2, (os.cpu_count() or 4) // 2)
SHOW_FIRST_N        = None   # None → all frames
RENDER_FIRST_N      = 300    # render overlays for first N frames only

# =======================
# Jake lane points (you can switch which one by changing JAKE_POINT)
# =======================
LANE_LEFT   = (300, 1340)
LANE_MID    = (490, 1340)
LANE_RIGHT  = (680, 1340)
JAKE_POINT  = LANE_MID  # pick one: LANE_LEFT / LANE_MID / LANE_RIGHT

# Lane target bearings (degrees from vertical; left=positive, right=negative)
LANE_TARGET_DEG = {
    "left":  -10.7,
    "mid":   +1.5,
    "right": +15.0,
}

def lane_name_from_point(p):
    if p == LANE_LEFT:  return "left"
    if p == LANE_MID:   return "mid"
    if p == LANE_RIGHT: return "right"
    # default to mid if unmatched
    return "mid"

# =======================
# System/Backends
# =======================
cv2.setUseOptimized(True)
try: cv2.setNumThreads(max(1, (os.cpu_count() or 1) - 1))
except Exception: pass

if torch.cuda.is_available():
    device, half = 0, True
    torch.backends.cudnn.benchmark = True
    try: torch.set_float32_matmul_precision('high')
    except Exception: pass
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model
# =======================
model = YOLO(weights)
try: model.fuse()
except Exception: pass

# Warmup
_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(_dummy, task="segment", imgsz=IMG_SIZE, device=device,
                  conf=CONF, iou=IOU, verbose=False, half=half, max_det=MAX_DET)

# =======================
# Precomputed
# =======================
TARGETS_BGR_F32 = np.array([(r,g,b)[::-1] for (r,g,b) in TARGET_COLORS_RGB], dtype=np.float32)
TOL2            = TOLERANCE * TOLERANCE

CLASS_COLOURS = {
    0:(255,255,0),1:(192,192,192),2:(0,128,255),3:(0,255,0),
    4:(255,0,255),5:(0,255,255),6:(255,128,0),7:(128,0,255),
    8:(0,0,128),9:(0,0,255),10:(128,128,0),11:(255,255,102)
}
LABELS = {
    0:"BOOTS",1:"GREYTRAIN",2:"HIGHBARRIER1",3:"JUMP",4:"LOWBARRIER1",
    5:"LOWBARRIER2",6:"ORANGETRAIN",7:"PILLAR",8:"RAMP",9:"RAILS",
    10:"SIDEWALK",11:"YELLOWTRAIN"
}

SAFE_GREEN = {9, 10}
WARN_YELLOW = {2,3,4,5,8}

# =======================
# Helpers
# =======================
def load_image_with_time(path: str):
    t0 = time.perf_counter()
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    t1 = time.perf_counter()
    return img, (t1 - t0) * 1000.0

def chunked(iterable, n):
    for i in range(0, len(iterable), n):
        yield iterable[i:i+n]

def highlight_rails_mask_only_fast(img_bgr, rail_mask):
    H, W = img_bgr.shape[:2]
    if not rail_mask.any():
        return np.zeros((H, W), dtype=bool)

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max()+1
    x0, x1 = xs.min(), xs.max()+1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - TARGETS_BGR_F32[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    colour_hit = np.any(dist2 <= TOL2, axis=-1)

    combined = np.logical_and(colour_hit, mask_roi)
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)
    if n <= 1:
        return np.zeros((H, W), dtype=bool)

    good = np.zeros_like(combined)
    areas  = stats[1:, cv2.CC_STAT_AREA]
    hs     = stats[1:, cv2.CC_STAT_HEIGHT]
    keep   = np.where((areas >= MIN_REGION_SIZE) & (hs >= MIN_REGION_HEIGHT))[0] + 1
    for k in keep:
        good[lbls == k] = True

    full = np.zeros((H, W), dtype=bool)
    full[y0:y1, x0:x1] = good
    return full

def red_vs_green_score(red_mask, green_mask):
    k = (HEAT_BLUR_KSIZE, HEAT_BLUR_KSIZE)
    r = cv2.blur(red_mask.astype(np.float32), k)
    g = cv2.blur(green_mask.astype(np.float32), k)
    diff = r - g
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2.0 * amax) + 0.5)
    return np.clip(norm * 255.0, 0, 255.0).astype(np.uint8)

def purple_triangles(score, H):
    top_ex = int(H * EXCLUDE_TOP_FRAC)
    bot_ex = int(H * EXCLUDE_BOTTOM_FRAC)

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    if top_ex: dark[:top_ex, :] = 0
    if bot_ex: dark[-bot_ex:, :] = 0

    dark = cv2.morphologyEx(
        dark, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9)),
        iterations=1
    )
    total_dark = int(dark.sum())
    if total_dark == 0:
        return [], None

    frac_thresh = int(np.ceil(MIN_DARK_FRACTION * total_dark))
    n_lbl, lbls, stats, _ = cv2.connectedComponentsWithStats(dark, 8)
    if n_lbl <= 1:
        return [], None

    tris = []
    for lbl in range(1, n_lbl):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area >= MIN_DARK_RED_AREA and area >= frac_thresh:
            ys, xs = np.where(lbls == lbl)
            if ys.size == 0:
                continue
            y_top = ys.min()
            x_mid = int(xs[ys == y_top].mean())
            tris.append((int(x_mid), int(y_top)))

    if not tris:
        return [], None

    best = min(tris, key=lambda xy: xy[1])
    return tris, best

def classify_triangles_at_sample(tri_positions, masks_np, classes_np, frame_H, frame_W, sample_up=SAMPLE_UP_PX):
    if masks_np is None or classes_np is None or len(tri_positions) == 0:
        return []
    mh, mw = masks_np.shape[1], masks_np.shape[2]
    sx = (mw - 1) / max(1, (frame_W - 1))
    sy = (mh - 1) / max(1, (frame_H - 1))
    colours = []
    for (x, y) in tri_positions:
        ys = max(0, y - sample_up)
        mx = int(round(x * sx)); my = int(round(ys * sy))
        if mx < 0: mx = 0
        elif mx >= mw: mx = mw - 1
        if my < 0: my = 0
        elif my >= mh: my = mh - 1
        cls_here = None
        for m, c in zip(masks_np, classes_np):
            if m[my, mx] > 0.5:
                cls_here = int(c); break
        if (cls_here is None) or (cls_here in SAFE_GREEN):
            colours.append(COLOR_GREEN)
        elif cls_here == 0:
            colours.append(COLOR_PINK)
        elif cls_here in WARN_YELLOW:
            colours.append(COLOR_YELLOW)
        else:
            colours.append(COLOR_RED)
    return colours

# Returns: (tri_best_xy, tri_count, mask_count, to_cpu_ms, post_ms, masks_np, classes_np, rail_mask, green, tri_positions, tri_colours)
def process_frame_post(frame_bgr, yolo_res):
    H, W = frame_bgr.shape[:2]
    if yolo_res.masks is None:
        return None, 0, 0, 0.0, 0.0, None, None, None, None, [], []

    t0_to_cpu = time.perf_counter()
    masks_np = yolo_res.masks.data.cpu().numpy()  # [n,h,w]
    mask_count = int(masks_np.shape[0])
    if hasattr(yolo_res.masks, "cls") and yolo_res.masks.cls is not None:
        classes_np = yolo_res.masks.cls.cpu().numpy().astype(int)
    else:
        classes_np = yolo_res.boxes.cls.cpu().numpy().astype(int)
    t1_to_cpu = time.perf_counter()
    to_cpu_ms = (t1_to_cpu - t0_to_cpu) * 1000.0

    if mask_count == 0 or classes_np.size == 0:
        return None, 0, mask_count, to_cpu_ms, 0.0, masks_np, classes_np, None, None, [], []

    rail_sel = (classes_np == RAIL_ID)
    if not np.any(rail_sel):
        return None, 0, mask_count, to_cpu_ms, 0.0, masks_np, classes_np, None, None, [], []

    t0_post = time.perf_counter()
    rail_masks = masks_np[rail_sel].astype(bool)        # [k,h,w]
    union = np.any(rail_masks, axis=0).astype(np.uint8) # [h,w]
    rail_mask = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST).astype(bool)

    green = highlight_rails_mask_only_fast(frame_bgr, rail_mask)
    red   = np.logical_and(rail_mask, np.logical_not(green))
    score = red_vs_green_score(red, green)
    tri_positions, tri_best = purple_triangles(score, H)

    # classify triangles by sampling masks above tip
    tri_colours = classify_triangles_at_sample(tri_positions, masks_np, classes_np, H, W, SAMPLE_UP_PX)

    t1_post = time.perf_counter()
    post_ms = (t1_post - t0_post) * 1000.0

    return tri_best, len(tri_positions), mask_count, to_cpu_ms, post_ms, masks_np, classes_np, rail_mask, green, tri_positions, tri_colours

def _colour_for_point(x, y, masks_np, classes_np, frame_H, frame_W):
    if masks_np is None or classes_np is None or masks_np.size == 0:
        return COLOR_GREEN
    mh, mw = masks_np.shape[1], masks_np.shape[2]
    sx = (mw - 1) / max(1, (frame_W - 1))
    sy = (mh - 1) / max(1, (frame_H - 1))
    mx = int(round(x * sx)); my = int(round(y * sy))
    if mx < 0: mx = 0
    elif mx >= mw: mx = mw - 1
    if my < 0: my = 0
    elif my >= mh: my = mh - 1
    cls_here = None
    for m, c in zip(masks_np, classes_np):
        if m[my, mx] > 0.5:
            cls_here = int(c); break
    if (cls_here is None) or (cls_here in SAFE_GREEN): return COLOR_GREEN
    if cls_here == 0: return COLOR_PINK
    if cls_here in WARN_YELLOW: return COLOR_YELLOW
    return COLOR_RED

def draw_triangle(img, x, y, size=TRI_SIZE_PX, colour=COLOR_RED):
    h = int(size * 1.2)
    pts = np.array([[x, y], [x-size, y+h], [x+size, y+h]], np.int32)
    cv2.fillConvexPoly(img, pts, colour)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, COLOR_BLACK, 1, cv2.LINE_AA)

def triangle_pts(x, y, size=TRI_SIZE_PX):
    h = int(size * 1.2)
    return np.array([[x, y], [x-size, y+h], [x+size, y+h]], np.int32)

# ===== Bearing-based Jake triangle selection =====
def signed_degrees_from_vertical(dx, dy):
    """
    Signed angle (degrees) from vertical (upwards).
    Conventions:
      - left of Jake (dx < 0) => +degrees
      - right of Jake (dx > 0) => -degrees
      - straight ahead (vertical) => ~0°
    Implementation: angle = -atan2(dx, -dy)  (since -dy>0 for triangles above)
    """
    if dx == 0 and dy == 0:
        return 0.0
    return -math.degrees(math.atan2(dx, -dy))

def select_triangle_by_bearing(tri_positions, jx, jy, target_deg, min_dy=6):
    """
    Pick the triangle ahead of Jake (yt < jy - min_dy) whose signed degrees-from-vertical
    is closest to the lane target.
    Returns (best_index, best_deg, best_err) or (-1, None, None) if none ahead.
    """
    best_i, best_deg, best_err = -1, None, None
    for i, (xt, yt) in enumerate(tri_positions):
        dx = xt - jx
        dy = yt - jy
        if dy >= -min_dy:  # requires triangle be above Jake by at least min_dy px
            continue
        deg = signed_degrees_from_vertical(dx, dy)
        err = abs(deg - target_deg)
        if (best_err is None) or (err < best_err):
            best_i, best_deg, best_err = i, deg, err
    return best_i, best_deg, best_err

# --- rendering (excluded from timing) ---
def render_overlays(frame_bgr, masks_np, classes_np, rail_mask, green_mask, tri_positions, tri_colours):
    """Draw all masks (class color) + labels, rail tint/green, coloured triangles,
       scout lines, and starburst lines to Jake's lane point. Highlight the triangle
       whose bearing is closest to the lane target."""
    out = frame_bgr.copy()
    H, W = out.shape[:2]
    alpha = 0.45

    # masks
    if masks_np is not None and classes_np is not None and masks_np.size:
        for m, c in zip(masks_np, classes_np):
            m_full = m
            if m.shape != (H, W):
                m_full = cv2.resize(m.astype(np.uint8), (W, H), interpolation=cv2.INTER_NEAREST).astype(bool)
            color = CLASS_COLOURS.get(int(c), (255,255,255))
            out[m_full] = (np.array(color, dtype=np.uint8) * alpha + out[m_full] * (1 - alpha)).astype(np.uint8)
            ys, xs = np.where(m_full)
            if xs.size:
                xc, yc = int(xs.mean()), int(ys.mean())
                label = LABELS.get(int(c), f"C{int(c)}")
                cv2.putText(out, label, (max(5, xc-40), max(20, yc)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLOR_BLACK, 2, cv2.LINE_AA)
                cv2.putText(out, label, (max(5, xc-40), max(20, yc)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)

    if rail_mask is not None:
        tint = out.copy()
        tint[rail_mask] = (0, 0, 255)
        out = cv2.addWeighted(tint, 0.30, out, 0.70, 0)
    if green_mask is not None:
        out[green_mask] = (0, 255, 0)

    # Scout lines
    if tri_positions:
        for (x, y) in tri_positions:
            y_end = max(0, y - SAMPLE_UP_PX)
            for yy in range(y, y_end - 1, -1):
                col = _colour_for_point(x, yy, masks_np, classes_np, H, W)
                out[yy, x] = col

    # Prepare lane target
    lane_name = lane_name_from_point(JAKE_POINT)
    target_deg = LANE_TARGET_DEG[lane_name]

    # Select best triangle by bearing
    xj, yj = JAKE_POINT
    best_idx, best_deg, _ = select_triangle_by_bearing(tri_positions, xj, yj, target_deg, min_dy=6)

    # Starburst lines + degree labels; highlight best
    for idx, (xt, yt) in enumerate(tri_positions):
        xt = max(0, min(W-1, int(xt)))
        yt = max(0, min(H-1, int(yt)))

        dx = xt - xj
        dy = yt - yj
        # degrees from vertical (signed as defined)
        deg_signed = signed_degrees_from_vertical(dx, dy)
        label = f"{deg_signed:.1f}°"

        line_color = COLOR_CYAN if idx == best_idx else COLOR_WHITE
        cv2.line(out, (xj, yj), (xt, yt), line_color, 2, cv2.LINE_AA)

        mid_x = int((xj + xt) / 2)
        mid_y = int((yj + yt) / 2)
        cv2.putText(out, label, (mid_x, mid_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, COLOR_BLACK, 2, cv2.LINE_AA)
        cv2.putText(out, label, (mid_x, mid_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255,255,255), 1, cv2.LINE_AA)

    # Triangles (draw after lines)
    for (x, y), col in zip(tri_positions, tri_colours):
        draw_triangle(out, x, y, colour=col)

    # Emphasize selected triangle + tag
    if best_idx is not None and 0 <= best_idx < len(tri_positions):
        xt, yt = tri_positions[best_idx]
        pts = triangle_pts(int(xt), int(yt), size=TRI_SIZE_PX)
        cv2.polylines(out, [pts.reshape(-1,1,2)], True, COLOR_CYAN, 3, cv2.LINE_AA)
        tag = f"JAKE_TRI ({lane_name}: target {target_deg:.1f}°)"
        cv2.putText(out, tag, (max(5, int(xt)-70), max(20, int(yt)-16)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, COLOR_BLACK, 2, cv2.LINE_AA)
        cv2.putText(out, tag, (max(5, int(xt)-70), max(20, int(yt)-16)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255,255,255), 1, cv2.LINE_AA)

    return out

# =======================
# Batched execution with prints; overlays saved for first N
# =======================
def run_pipeline_with_prints_and_overlays():
    paths = (
        glob.glob(str(frames_dir/"frame_*.jpg")) +
        glob.glob(str(frames_dir/"frame_*.png")) +
        glob.glob(str(frames_dir/"*.jpg")) +
        glob.glob(str(frames_dir/"*.png"))
    )
    paths = sorted(set(paths))
    if not paths:
        raise FileNotFoundError(f"No images in: {frames_dir}")
    if SHOW_FIRST_N is not None:
        paths = paths[:SHOW_FIRST_N]

    N = len(paths)
    results_triangle_xy = [None] * N

    def load_batch(batch_paths):
        imgs = [None] * len(batch_paths)
        read_ms = [0.0] * len(batch_paths)
        with ThreadPoolExecutor(max_workers=THREADS_IO) as ex:
            fut2idx = {ex.submit(load_image_with_time, p): i for i, p in enumerate(batch_paths)}
            for fut in as_completed(fut2idx):
                i = fut2idx[fut]
                img, r_ms = fut.result()
                imgs[i] = img
                read_ms[i] = r_ms
        ok = [(p, im, rm) for p, im, rm in zip(batch_paths, imgs, read_ms) if im is not None]
        if not ok:
            return [], [], []
        b_paths, b_imgs, b_read = zip(*ok)
        return list(b_paths), list(b_imgs), list(b_read)

    idx_global = 0
    for batch_paths in chunked(paths, BATCH):
        batch_paths, imgs_bgr, read_ms_list = load_batch(batch_paths)
        B = len(imgs_bgr)
        if B == 0:
            idx_global += len(batch_paths)
            continue

        t0_inf = time.perf_counter()
        res_list = model.predict(
            imgs_bgr, task="segment", imgsz=IMG_SIZE, device=device,
            conf=CONF, iou=IOU, verbose=False, half=half, max_det=MAX_DET,
            batch=B
        )
        try:
            if device == 0 and torch.cuda.is_available():
                torch.cuda.synchronize()
            elif device == "mps" and getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
                torch.mps.synchronize()
        except Exception:
            pass
        t1_inf = time.perf_counter()
        infer_ms_share = ((t1_inf - t0_inf) * 1000.0) / B

        for j, (img, yres, read_ms) in enumerate(zip(imgs_bgr, res_list, read_ms_list)):
            (tri_best_xy, tri_count, mask_count, to_cpu_ms, post_ms,
             masks_np, classes_np, rail_mask, green_mask, tri_positions, tri_colours) = process_frame_post(img, yres)

            results_triangle_xy[idx_global + j] = tri_best_xy
            proc_ms = infer_ms_share + to_cpu_ms + post_ms
            fname = os.path.basename(batch_paths[j])
            frame_idx = idx_global + j + 1

            print(f"[{frame_idx}/{N}] {fname}  "
                  f"read {read_ms:.1f} | infer {infer_ms_share:.1f} | "
                  f"to_cpu {to_cpu_ms:.1f} | post {post_ms:.1f} | "
                  f"masks {mask_count} | triangles {tri_count} "
                  f"=> proc {proc_ms:.1f} ms")

            if frame_idx <= RENDER_FIRST_N:
                overlay = render_overlays(img, masks_np, classes_np, rail_mask, green_mask, tri_positions, tri_colours)
                out_path = out_dir / f"overlay_{frame_idx:04d}_{fname}"
                cv2.imwrite(str(out_path), overlay)

        idx_global += B

    return results_triangle_xy

# =======================
# Entry
# =======================
if __name__ == "__main__":
    _ = run_pipeline_with_prints_and_overlays()


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs
[1/312] frame_00000.png  read 37.0 | infer 82.8 | to_cpu 0.6 | post 0.0 | masks 1 | triangles 0 => proc 83.4 ms
[2/312] frame_00001.png  read 35.9 | infer 49.4 | to_cpu 0.9 | post 66.5 | masks 3 | triangles 1 => proc 116.9 ms
[3/312] frame_00002.png  read 39.6 | infer 43.3 | to_cpu 0.9 | post 100.6 | masks 3 | triangles 3 => proc 144.7 ms
[4/312] frame_00003.png  read 37.2 | infer 42.2 | to_cpu 0.9 | post 112.9 | masks 3 | triangles 2 => proc 156.0 ms
[5/312] frame_00004.png  read 40.6 | infer 43.7 | to_cpu 1.7 | post 159.4 | masks 4 | triangles 1 => proc 204.9 ms
[6/312] frame_00005.png  read 37.3 | infer 46.7 | to_cpu 1.0 | post 158.1 | masks 5 | triangles 1 => proc 205.8 ms
[7/312] frame_00006.png  read 39.8 | infer 34.3 | to_cpu 1.1 | post 155.1 | masks 5 | triangles 2 => proc 190.6 ms
[8/312] frame_00007.png  read 37.2 | infer 39.0 | to_cpu 0.8 | post 160.7 | masks 4 | triangles 2 => proc 200.5

In [ ]:
#Curved search line for Npxls ahead at n1-4 degrees from vertical 

In [11]:
#!/usr/bin/env python3
# Ultra-fast, batched pipeline with threaded I/O + per-frame timing & counts
# Renders ALL masks + triangles coloured by the mask sampled N px along a ray
# + Scout lines (render only)
# + Starburst lines from each triangle tip to Jake's lane point
# + Lane-bearing selector for Jake's triangle
# + Lane-aware angled sampling for mask lookup with per-ray degree labels

import os, glob, sys, time, math
import cv2, torch, numpy as np
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from ultralytics import YOLO

# =======================
# Config
# =======================
home       = os.path.expanduser("~")
weights    = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
frames_dir = Path(home) / "Documents" / "GitHub" / "Ai-plays-SubwaySurfers" / "frames"

# SAVE HERE
out_dir    = Path(home) / "Documents" / "GitHub" / "Ai-plays-SubwaySurfers" / "out_overlays6"
out_dir.mkdir(parents=True, exist_ok=True)

RAIL_ID    = 9
IMG_SIZE   = 512
CONF, IOU  = 0.30, 0.45
MAX_DET    = 30

# Color/region filter
TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

# Heat/triangle
HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
MIN_DARK_FRACTION   = 0.15
TRI_SIZE_PX         = 18

# Sampling ray length (pixels in frame space)
SAMPLE_UP_PX        = 120

# ===== BEND PARAMETERS (degrees; + = lean LEFT, - = lean RIGHT)
# If JAKE_POINT = LANE_LEFT, triangles to the RIGHT bend LEFT by N1
BEND_LEFT_STATE_RIGHT_DEG  = +20.0  # N1
# If JAKE_POINT = LANE_MID, triangles RIGHT of JakeTri bend LEFT by N2
BEND_MID_STATE_RIGHT_DEG   = +20.0  # N2
# If JAKE_POINT = LANE_MID, triangles LEFT of JakeTri bend RIGHT by N3
BEND_MID_STATE_LEFT_DEG    = -20.0  # N3
# If JAKE_POINT = LANE_RIGHT, triangles to the LEFT bend RIGHT by N4
BEND_RIGHT_STATE_LEFT_DEG  = -20.0  # N4

# Colours (BGR)
COLOR_GREEN  = (0, 255, 0)
COLOR_PINK   = (203, 192, 255)
COLOR_YELLOW = (0, 255, 255)
COLOR_RED    = (0, 0, 255)
COLOR_WHITE  = (255, 255, 255)
COLOR_CYAN   = (255, 255, 0)
COLOR_BLACK  = (0, 0, 0)

# Runtime
BATCH               = 1
THREADS_IO          = max(2, (os.cpu_count() or 4) // 2)
SHOW_FIRST_N        = None   # None → all frames
RENDER_FIRST_N      = 300    # render overlays for first N frames only

# =======================
# Jake lane points
# =======================
LANE_LEFT   = (300, 1340)
LANE_MID    = (490, 1340)
LANE_RIGHT  = (680, 1340)
JAKE_POINT  = LANE_MID  # pick one: LANE_LEFT / LANE_MID / LANE_RIGHT

# Lane target bearings (degrees from vertical; left=positive, right=negative)
LANE_TARGET_DEG = {
    "left":  -10.7,
    "mid":   +1.5,
    "right": +15.0,
}

def lane_name_from_point(p):
    if p == LANE_LEFT:  return "left"
    if p == LANE_MID:   return "mid"
    if p == LANE_RIGHT: return "right"
    return "mid"

# =======================
# System/Backends
# =======================
cv2.setUseOptimized(True)
try: cv2.setNumThreads(max(1, (os.cpu_count() or 1) - 1))
except Exception: pass

if torch.cuda.is_available():
    device, half = 0, True
    torch.backends.cudnn.benchmark = True
    try: torch.set_float32_matmul_precision('high')
    except Exception: pass
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model
# =======================
model = YOLO(weights)
try: model.fuse()
except Exception: pass

# Warmup
_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(_dummy, task="segment", imgsz=IMG_SIZE, device=device,
                  conf=CONF, iou=IOU, verbose=False, half=half, max_det=MAX_DET)

# =======================
# Precomputed
# =======================
TARGETS_BGR_F32 = np.array([(r,g,b)[::-1] for (r,g,b) in TARGET_COLORS_RGB], dtype=np.float32)
TOL2            = TOLERANCE * TOLERANCE

CLASS_COLOURS = {
    0:(255,255,0),1:(192,192,192),2:(0,128,255),3:(0,255,0),
    4:(255,0,255),5:(0,255,255),6:(255,128,0),7:(128,0,255),
    8:(0,0,128),9:(0,0,255),10:(128,128,0),11:(255,255,102)
}
LABELS = {
    0:"BOOTS",1:"GREYTRAIN",2:"HIGHBARRIER1",3:"JUMP",4:"LOWBARRIER1",
    5:"LOWBARRIER2",6:"ORANGETRAIN",7:"PILLAR",8:"RAMP",9:"RAILS",
    10:"SIDEWALK",11:"YELLOWTRAIN"
}

SAFE_GREEN = {9, 10}
WARN_YELLOW = {2,3,4,5,8}

# =======================
# Helpers
# =======================
def load_image_with_time(path: str):
    t0 = time.perf_counter()
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    t1 = time.perf_counter()
    return img, (t1 - t0) * 1000.0

def chunked(iterable, n):
    for i in range(0, len(iterable), n):
        yield iterable[i:i+n]

def highlight_rails_mask_only_fast(img_bgr, rail_mask):
    H, W = img_bgr.shape[:2]
    if not rail_mask.any():
        return np.zeros((H, W), dtype=bool)

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max()+1
    x0, x1 = xs.min(), xs.max()+1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - TARGETS_BGR_F32[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    colour_hit = np.any(dist2 <= TOL2, axis=-1)

    combined = np.logical_and(colour_hit, mask_roi)
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)
    if n <= 1:
        return np.zeros((H, W), dtype=bool)

    good = np.zeros_like(combined)
    areas  = stats[1:, cv2.CC_STAT_AREA]
    hs     = stats[1:, cv2.CC_STAT_HEIGHT]
    keep   = np.where((areas >= MIN_REGION_SIZE) & (hs >= MIN_REGION_HEIGHT))[0] + 1
    for k in keep:
        good[lbls == k] = True

    full = np.zeros((H, W), dtype=bool)
    full[y0:y1, x0:x1] = good
    return full

def red_vs_green_score(red_mask, green_mask):
    k = (HEAT_BLUR_KSIZE, HEAT_BLUR_KSIZE)
    r = cv2.blur(red_mask.astype(np.float32), k)
    g = cv2.blur(green_mask.astype(np.float32), k)
    diff = r - g
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2.0 * amax) + 0.5)
    return np.clip(norm * 255.0, 0, 255.0).astype(np.uint8)

def purple_triangles(score, H):
    top_ex = int(H * EXCLUDE_TOP_FRAC)
    bot_ex = int(H * EXCLUDE_BOTTOM_FRAC)

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    if top_ex: dark[:top_ex, :] = 0
    if bot_ex: dark[-bot_ex:, :] = 0

    dark = cv2.morphologyEx(
        dark, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9)),
        iterations=1
    )
    total_dark = int(dark.sum())
    if total_dark == 0:
        return [], None

    frac_thresh = int(np.ceil(MIN_DARK_FRACTION * total_dark))
    n_lbl, lbls, stats, _ = cv2.connectedComponentsWithStats(dark, 8)
    if n_lbl <= 1:
        return [], None

    tris = []
    for lbl in range(1, n_lbl):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area >= MIN_DARK_RED_AREA and area >= frac_thresh:
            ys, xs = np.where(lbls == lbl)
            if ys.size == 0:
                continue
            y_top = ys.min()
            x_mid = int(xs[ys == y_top].mean())
            tris.append((int(x_mid), int(y_top)))

    if not tris:
        return [], None

    best = min(tris, key=lambda xy: xy[1])
    return tris, best

# ===== Angled-bend helper =====
def pick_bend_angle(jake_point, xt, x_ref, idx, best_idx):
    """Return θ (deg from vertical; +left, -right) for sampling.
       Jake's triangle (best_idx) always returns 0°."""
    if idx == best_idx:
        return 0.0

    if jake_point == LANE_LEFT:
        # Triangles to the RIGHT of Jake bend LEFT by N1
        return BEND_LEFT_STATE_RIGHT_DEG if xt > x_ref else 0.0

    if jake_point == LANE_RIGHT:
        # Triangles to the LEFT of Jake bend RIGHT by N4
        return BEND_RIGHT_STATE_LEFT_DEG if xt < x_ref else 0.0

    # Middle lane: use JakeTri's x as the divider; right bends LEFT (N2), left bends RIGHT (N3)
    if xt > x_ref:
        return BEND_MID_STATE_RIGHT_DEG
    elif xt < x_ref:
        return BEND_MID_STATE_LEFT_DEG
    return 0.0

# ===== Bearing-based Jake triangle selection =====
def signed_degrees_from_vertical(dx, dy):
    if dx == 0 and dy == 0:
        return 0.0
    return -math.degrees(math.atan2(dx, -dy))

def select_triangle_by_bearing(tri_positions, jx, jy, target_deg, min_dy=6):
    best_i, best_deg, best_err = -1, None, None
    for i, (xt, yt) in enumerate(tri_positions):
        dx = xt - jx
        dy = yt - jy
        if dy >= -min_dy:  # must be above Jake
            continue
        deg = signed_degrees_from_vertical(dx, dy)
        err = abs(deg - target_deg)
        if (best_err is None) or (err < best_err):
            best_i, best_deg, best_err = i, deg, err
    return best_i, best_deg, best_err

# ===== UPDATED: triangle classification using angled sampling =====
def classify_triangles_at_sample_curved(
    tri_positions, masks_np, classes_np, frame_H, frame_W,
    jake_point, x_ref, best_idx, sample_px=SAMPLE_UP_PX
):
    """
    For each triangle tip (x,y), sample masks at a point N px away along a lane-aware
    angled ray. Returns:
      colours: list of BGR colours per triangle
      rays:    list of ((x0,y0),(x1,y1),theta_deg) for debug drawing/labels
    """
    if masks_np is None or classes_np is None or len(tri_positions) == 0:
        return [], []

    mh, mw = masks_np.shape[1], masks_np.shape[2]
    sx = (mw - 1) / max(1, (frame_W - 1))
    sy = (mh - 1) / max(1, (frame_H - 1))

    colours, rays = [], []

    for idx, (x, y) in enumerate(tri_positions):
        # Choose bend angle (Jake tri = 0°, others per lane rules)
        theta = pick_bend_angle(jake_point, x, x_ref, idx, best_idx)

        # Direction from vertical (0,-1) rotated by theta (deg)
        rad = math.radians(theta)
        dx = math.sin(rad)          # +left, -right
        dy = -math.cos(rad)         # up

        xs = int(round(x + dx * sample_px))
        ys = int(round(y + dy * sample_px))

        # Clamp to frame
        xs = max(0, min(frame_W - 1, xs))
        ys = max(0, min(frame_H - 1, ys))

        # Map to mask space
        mx = int(round(xs * sx))
        my = int(round(ys * sy))
        mx = 0 if mx < 0 else (mw - 1 if mx >= mw else mx)
        my = 0 if my < 0 else (mh - 1 if my >= mh else my)

        # Class at sample point
        cls_here = None
        for m, c in zip(masks_np, classes_np):
            if m[my, mx] > 0.5:
                cls_here = int(c)
                break

        if (cls_here is None) or (cls_here in SAFE_GREEN):
            colours.append(COLOR_GREEN)
        elif cls_here == 0:
            colours.append(COLOR_PINK)
        elif cls_here in WARN_YELLOW:
            colours.append(COLOR_YELLOW)
        else:
            colours.append(COLOR_RED)

        rays.append(((int(x), int(y)), (xs, ys), float(theta)))

    return colours, rays

# Returns: (..., tri_positions, tri_colours, tri_rays, best_idx, best_deg, x_ref)
def process_frame_post(frame_bgr, yolo_res):
    H, W = frame_bgr.shape[:2]
    if yolo_res.masks is None:
        return None, 0, 0, 0.0, 0.0, None, None, None, None, [], [], [], None, None, None

    t0_to_cpu = time.perf_counter()
    masks_np = yolo_res.masks.data.cpu().numpy()  # [n,h,w]
    mask_count = int(masks_np.shape[0])
    if hasattr(yolo_res.masks, "cls") and yolo_res.masks.cls is not None:
        classes_np = yolo_res.masks.cls.cpu().numpy().astype(int)
    else:
        classes_np = yolo_res.boxes.cls.cpu().numpy().astype(int)
    t1_to_cpu = time.perf_counter()
    to_cpu_ms = (t1_to_cpu - t0_to_cpu) * 1000.0

    if mask_count == 0 or classes_np.size == 0:
        return None, 0, mask_count, to_cpu_ms, 0.0, masks_np, classes_np, None, None, [], [], [], None, None, None

    rail_sel = (classes_np == RAIL_ID)
    if not np.any(rail_sel):
        return None, 0, mask_count, to_cpu_ms, 0.0, masks_np, classes_np, None, None, [], [], [], None, None, None

    t0_post = time.perf_counter()
    rail_masks = masks_np[rail_sel].astype(bool)        # [k,h,w]
    union = np.any(rail_masks, axis=0).astype(np.uint8) # [h,w]
    rail_mask = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST).astype(bool)

    green = highlight_rails_mask_only_fast(frame_bgr, rail_mask)
    red   = np.logical_and(rail_mask, np.logical_not(green))
    score = red_vs_green_score(red, green)
    tri_positions, tri_best = purple_triangles(score, H)

    # --- decide Jake triangle first (bearing vs target)
    lane_name = lane_name_from_point(JAKE_POINT)
    target_deg = LANE_TARGET_DEG[lane_name]
    xj, yj = JAKE_POINT
    best_idx, best_deg, _ = select_triangle_by_bearing(tri_positions, xj, yj, target_deg, min_dy=6)

    # x reference for bending:
    # mid lane -> x of JakeTri if found, else Jake point x; side lanes -> Jake point x
    if lane_name == "mid" and best_idx is not None and 0 <= best_idx < len(tri_positions):
        x_ref = tri_positions[best_idx][0]
    else:
        x_ref = xj

    # classify with angled sampling (JakeTri ray = 0°)
    tri_colours, tri_rays = classify_triangles_at_sample_curved(
        tri_positions, masks_np, classes_np, H, W, JAKE_POINT, x_ref, best_idx, SAMPLE_UP_PX
    )

    t1_post = time.perf_counter()
    post_ms = (t1_post - t0_post) * 1000.0

    return tri_best, len(tri_positions), mask_count, to_cpu_ms, post_ms, masks_np, classes_np, rail_mask, green, tri_positions, tri_colours, tri_rays, best_idx, best_deg, x_ref

def _colour_for_point(x, y, masks_np, classes_np, frame_H, frame_W):
    if masks_np is None or classes_np is None or masks_np.size == 0:
        return COLOR_GREEN
    mh, mw = masks_np.shape[1], masks_np.shape[2]
    sx = (mw - 1) / max(1, (frame_W - 1))
    sy = (mh - 1) / max(1, (frame_H - 1))
    mx = int(round(x * sx)); my = int(round(y * sy))
    mx = 0 if mx < 0 else (mw - 1 if mx >= mw else mx)
    my = 0 if my < 0 else (mh - 1 if my >= mh else my)
    cls_here = None
    for m, c in zip(masks_np, classes_np):
        if m[my, mx] > 0.5:
            cls_here = int(c); break
    if (cls_here is None) or (cls_here in SAFE_GREEN): return COLOR_GREEN
    if cls_here == 0: return COLOR_PINK
    if cls_here in WARN_YELLOW: return COLOR_YELLOW
    return COLOR_RED

def draw_triangle(img, x, y, size=TRI_SIZE_PX, colour=COLOR_RED):
    h = int(size * 1.2)
    pts = np.array([[x, y], [x-size, y+h], [x+size, y+h]], np.int32)
    cv2.fillConvexPoly(img, pts, colour)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, COLOR_BLACK, 1, cv2.LINE_AA)

def triangle_pts(x, y, size=TRI_SIZE_PX):
    h = int(size * 1.2)
    return np.array([[x, y], [x-size, y+h], [x+size, y+h]], np.int32)

# --- rendering (excluded from timing) ---
def render_overlays(frame_bgr, masks_np, classes_np, rail_mask, green_mask,
                    tri_positions, tri_colours, tri_rays, best_idx, best_deg, x_ref):
    out = frame_bgr.copy()
    H, W = out.shape[:2]
    alpha = 0.45

    # masks
    if masks_np is not None and classes_np is not None and masks_np.size:
        for m, c in zip(masks_np, classes_np):
            m_full = m
            if m.shape != (H, W):
                m_full = cv2.resize(m.astype(np.uint8), (W, H), interpolation=cv2.INTER_NEAREST).astype(bool)
            color = CLASS_COLOURS.get(int(c), (255,255,255))
            out[m_full] = (np.array(color, dtype=np.uint8) * alpha + out[m_full] * (1 - alpha)).astype(np.uint8)
            ys, xs = np.where(m_full)
            if xs.size:
                xc, yc = int(xs.mean()), int(ys.mean())
                label = LABELS.get(int(c), f"C{int(c)}")
                cv2.putText(out, label, (max(5, xc-40), max(20, yc)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLOR_BLACK, 2, cv2.LINE_AA)
                cv2.putText(out, label, (max(5, xc-40), max(20, yc)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)

    if rail_mask is not None:
        tint = out.copy()
        tint[rail_mask] = (0, 0, 255)
        out = cv2.addWeighted(tint, 0.30, out, 0.70, 0)
    if green_mask is not None:
        out[green_mask] = (0, 255, 0)

    # Scout lines (tiny)
    if tri_positions:
        for (x, y) in tri_positions:
            y_end = max(0, y - SAMPLE_UP_PX)
            for yy in range(y, y_end - 1, -1):
                col = _colour_for_point(x, yy, masks_np, classes_np, H, W)
                out[yy, x] = col

    # Starburst lines to Jake with degree labels
    xj, yj = JAKE_POINT
    for idx, (xt, yt) in enumerate(tri_positions):
        xt = max(0, min(W-1, int(xt)))
        yt = max(0, min(H-1, int(yt)))
        dx = xt - xj; dy = yt - yj
        deg_signed = signed_degrees_from_vertical(dx, dy)
        line_color = COLOR_CYAN if idx == best_idx else COLOR_WHITE
        cv2.line(out, (xj, yj), (xt, yt), line_color, 2, cv2.LINE_AA)
        mid_x = int((xj + xt) / 2); mid_y = int((yj + yt) / 2)
        cv2.putText(out, f"{deg_signed:.1f}°", (mid_x, mid_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, COLOR_BLACK, 2, cv2.LINE_AA)
        cv2.putText(out, f"{deg_signed:.1f}°", (mid_x, mid_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255,255,255), 1, cv2.LINE_AA)

    # Draw angled sampling rays + bend-degree labels (JakeTri shows 0.0°)
    for (p0, p1, theta) in tri_rays:
        cv2.line(out, p0, p1, (255,255,255), 2, cv2.LINE_AA)
        mx = (p0[0] + p1[0]) // 2
        my = (p0[1] + p1[1]) // 2
        txt = f"{theta:+.1f}°"
        cv2.putText(out, txt, (mx, my),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, COLOR_BLACK, 2, cv2.LINE_AA)
        cv2.putText(out, txt, (mx, my),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255,255,255), 1, cv2.LINE_AA)

    # Triangles after lines
    for (x, y), col in zip(tri_positions, tri_colours):
        draw_triangle(out, x, y, colour=col)

    # Emphasize selected triangle + tag
    lane_name = lane_name_from_point(JAKE_POINT)
    target_deg = LANE_TARGET_DEG[lane_name]
    if best_idx is not None and 0 <= best_idx < len(tri_positions):
        xt, yt = tri_positions[best_idx]
        pts = triangle_pts(int(xt), int(yt), size=TRI_SIZE_PX)
        cv2.polylines(out, [pts.reshape(-1,1,2)], True, COLOR_CYAN, 3, cv2.LINE_AA)
        tag = f"JAKE_TRI ({lane_name}: target {target_deg:.1f}°)"
        cv2.putText(out, tag, (max(5, int(xt)-70), max(20, int(yt)-16)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, COLOR_BLACK, 2, cv2.LINE_AA)
        cv2.putText(out, tag, (max(5, int(xt)-70), max(20, int(yt)-16)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255,255,255), 1, cv2.LINE_AA)

    return out

# =======================
# Batched execution with prints; overlays saved for first N
# =======================
def run_pipeline_with_prints_and_overlays():
    paths = (
        glob.glob(str(frames_dir/"frame_*.jpg")) +
        glob.glob(str(frames_dir/"frame_*.png")) +
        glob.glob(str(frames_dir/"*.jpg")) +
        glob.glob(str(frames_dir/"*.png"))
    )
    paths = sorted(set(paths))
    if not paths:
        raise FileNotFoundError(f"No images in: {frames_dir}")
    if SHOW_FIRST_N is not None:
        paths = paths[:SHOW_FIRST_N]

    N = len(paths)
    results_triangle_xy = [None] * N

    def load_batch(batch_paths):
        imgs = [None] * len(batch_paths)
        read_ms = [0.0] * len(batch_paths)
        with ThreadPoolExecutor(max_workers=THREADS_IO) as ex:
            fut2idx = {ex.submit(load_image_with_time, p): i for i, p in enumerate(batch_paths)}
            for fut in as_completed(fut2idx):
                i = fut2idx[fut]
                img, r_ms = fut.result()
                imgs[i] = img
                read_ms[i] = r_ms
        ok = [(p, im, rm) for p, im, rm in zip(batch_paths, imgs, read_ms) if im is not None]
        if not ok:
            return [], [], []
        b_paths, b_imgs, b_read = zip(*ok)
        return list(b_paths), list(b_imgs), list(b_read)

    idx_global = 0
    for batch_paths in chunked(paths, BATCH):
        batch_paths, imgs_bgr, read_ms_list = load_batch(batch_paths)
        B = len(imgs_bgr)
        if B == 0:
            idx_global += len(batch_paths)
            continue

        t0_inf = time.perf_counter()
        res_list = model.predict(
            imgs_bgr, task="segment", imgsz=IMG_SIZE, device=device,
            conf=CONF, iou=IOU, verbose=False, half=half, max_det=MAX_DET,
            batch=B
        )
        try:
            if device == 0 and torch.cuda.is_available():
                torch.cuda.synchronize()
            elif device == "mps" and getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
                torch.mps.synchronize()
        except Exception:
            pass
        t1_inf = time.perf_counter()
        infer_ms_share = ((t1_inf - t0_inf) * 1000.0) / B

        for j, (img, yres, read_ms) in enumerate(zip(imgs_bgr, res_list, read_ms_list)):
            (tri_best_xy, tri_count, mask_count, to_cpu_ms, post_ms,
             masks_np, classes_np, rail_mask, green_mask, tri_positions, tri_colours,
             tri_rays, best_idx, best_deg, x_ref) = process_frame_post(img, yres)

            results_triangle_xy[idx_global + j] = tri_best_xy
            proc_ms = infer_ms_share + to_cpu_ms + post_ms
            fname = os.path.basename(batch_paths[j])
            frame_idx = idx_global + j + 1

            print(f"[{frame_idx}/{N}] {fname}  "
                  f"read {read_ms:.1f} | infer {infer_ms_share:.1f} | "
                  f"to_cpu {to_cpu_ms:.1f} | post {post_ms:.1f} | "
                  f"masks {mask_count} | triangles {tri_count} "
                  f"=> proc {proc_ms:.1f} ms")

            if frame_idx <= RENDER_FIRST_N:
                overlay = render_overlays(img, masks_np, classes_np, rail_mask, green_mask,
                                          tri_positions, tri_colours, tri_rays, best_idx, best_deg, x_ref)
                out_path = out_dir / f"overlay_{frame_idx:04d}_{fname}"
                cv2.imwrite(str(out_path), overlay)

        idx_global += B

    return results_triangle_xy

# =======================
# Entry
# =======================
if __name__ == "__main__":
    _ = run_pipeline_with_prints_and_overlays()


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs
[1/312] frame_00000.png  read 45.1 | infer 209.5 | to_cpu 1.0 | post 0.0 | masks 1 | triangles 0 => proc 210.5 ms
[2/312] frame_00001.png  read 39.8 | infer 109.4 | to_cpu 0.8 | post 90.8 | masks 3 | triangles 1 => proc 201.1 ms
[3/312] frame_00002.png  read 38.8 | infer 48.4 | to_cpu 0.7 | post 106.0 | masks 3 | triangles 3 => proc 155.2 ms
[4/312] frame_00003.png  read 39.1 | infer 62.7 | to_cpu 0.8 | post 114.3 | masks 3 | triangles 2 => proc 177.8 ms
[5/312] frame_00004.png  read 41.2 | infer 62.9 | to_cpu 0.8 | post 172.1 | masks 4 | triangles 1 => proc 235.9 ms
[6/312] frame_00005.png  read 54.9 | infer 184.7 | to_cpu 2.2 | post 239.7 | masks 5 | triangles 1 => proc 426.6 ms
[7/312] frame_00006.png  read 41.1 | infer 61.2 | to_cpu 1.4 | post 164.8 | masks 5 | triangles 2 => proc 227.3 ms
[8/312] frame_00007.png  read 40.3 | infer 43.9 | to_cpu 0.7 | post 177.9 | masks 4 | triangles 2 => proc 2

In [29]:
#Hardcoded jake lane state without prints full run

In [3]:
#!/usr/bin/env python3
# Ultra-fast, single-frame pipeline with per-frame timing & counts
# Headless/speed mode: NO overlays, NO disk writes — just detection + timing prints.
# Notebook-stable build: aggressive env/thread guards + safe device selection.

# =========================
# Stability ENV (MUST be set BEFORE imports of torch/cv2/numpy)
# =========================
import os
# General thread taming (Jupyter-friendly)
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")
os.environ.setdefault("KMP_DUPLICATE_LIB_OK", "TRUE")   # avoid libomp conflicts on macOS

# OpenCV: disable OpenCL (often crashes on macOS)
os.environ.setdefault("OPENCV_OPENCL_DEVICE", "disabled")
os.environ.setdefault("OPENCV_OPENCL_RUNTIME", "disabled")

# PyTorch MPS (Metal) guards — only honored if MPS is actually used
os.environ.setdefault("PYTORCH_ENABLE_MPS_FALLBACK", "1")      # keep MPS alive if an op is missing
os.environ.setdefault("PYTORCH_MPS_HIGH_WATERMARK_RATIO", "0.0")  # be conservative with memory

# Ultralytics: avoid settings writes (less I/O/threads in notebooks)
os.environ.setdefault("ULTRALYTICS_SETTINGS_RESET", "1")

# =========================
# Imports
# =========================
import glob, sys, time, math
import cv2, torch, numpy as np
from pathlib import Path
from ultralytics import YOLO

# =======================
# Config
# =======================
home       = os.path.expanduser("~")
weights    = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
frames_dir = Path(home) / "Documents" / "GitHub" / "Ai-plays-SubwaySurfers" / "frames"

RAIL_ID    = 9
IMG_SIZE   = 512
CONF, IOU  = 0.30, 0.45
MAX_DET    = 30

# Color/region filter (rails-green region finder)
TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

# Heat/triangle
HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
MIN_DARK_FRACTION   = 0.15

# Triangle tip sampling distance (kept for parity; not used in headless prints)
SAMPLE_UP_PX        = 65

# Runtime
SHOW_FIRST_N        = None   # None → all frames

# =======================
# Jake lane points + lane targets (kept for selection logic)
# =======================
LANE_LEFT   = (300, 1340)
LANE_MID    = (490, 1340)
LANE_RIGHT  = (680, 1340)
JAKE_POINT  = LANE_RIGHT  # pick one: LANE_LEFT / LANE_MID / LANE_RIGHT

# Lane target bearings (signed degrees from vertical; left=positive, right=negative)
LANE_TARGET_DEG = {
    "left":  -10.7,
    "mid":   +1.5,
    "right": +15.0,
}

def lane_name_from_point(p):
    if p == LANE_LEFT:  return "left"
    if p == LANE_MID:   return "mid"
    if p == LANE_RIGHT: return "right"
    return "mid"

# =======================
# System/Backends (Jupyter-safe threading + device choice)
# =======================
def _running_in_notebook() -> bool:
    try:
        from IPython import get_ipython  # noqa
        return get_ipython() is not None
    except Exception:
        return False

# OpenCV: keep threads to 1 in notebooks to avoid crashes
cv2.setUseOptimized(True)
try:
    cv2.setNumThreads(1 if _running_in_notebook() else max(1, (os.cpu_count() or 1) - 1))
except Exception:
    pass

# Device selection:
# - In notebooks on macOS, MPS can hard-abort inside ipykernel threads.
#   Default to CPU in notebooks unless ALLOW_MPS_IN_NOTEBOOK=1 is set.
ALLOW_MPS_NOTEBOOK = os.environ.get("ALLOW_MPS_IN_NOTEBOOK", "0") == "1"

if torch.cuda.is_available():
    device, half = 0, True
    torch.backends.cudnn.benchmark = True
    try: torch.set_float32_matmul_precision('high')
    except Exception: pass
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available() and (not _running_in_notebook() or ALLOW_MPS_NOTEBOOK):
    device, half = "mps", False
else:
    device, half = "cpu", False

# Torch threads: be conservative in notebooks
try:
    if _running_in_notebook():
        torch.set_num_threads(1)
        if hasattr(torch, "set_num_interop_threads"):
            torch.set_num_interop_threads(1)
except Exception:
    pass

# =======================
# Model
# =======================
model = YOLO(weights)
try: model.fuse()
except Exception: pass

# Warmup (kept small; no hard sync)
_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
with torch.no_grad():
    _ = model.predict(_dummy, task="segment", imgsz=IMG_SIZE, device=device,
                      conf=CONF, iou=IOU, verbose=False, half=half, max_det=MAX_DET)

# =======================
# Precomputed
# =======================
TARGETS_BGR_F32 = np.array([(r,g,b)[::-1] for (r,g,b) in TARGET_COLORS_RGB], dtype=np.float32)
TOL2            = TOLERANCE * TOLERANCE
MORPH_SE        = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9))

# =======================
# Helpers
# =======================
def load_image_with_time(path: str):
    t0 = time.perf_counter()
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    t1 = time.perf_counter()
    return img, (t1 - t0) * 1000.0

def highlight_rails_mask_only_fast(img_bgr, rail_mask):
    H, W = img_bgr.shape[:2]
    if rail_mask is None or not rail_mask.any():
        return np.zeros((H, W), dtype=bool)

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max()+1
    x0, x1 = xs.min(), xs.max()+1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - TARGETS_BGR_F32[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    colour_hit = np.any(dist2 <= TOL2, axis=-1)

    comp = (colour_hit & mask_roi).astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)
    if n <= 1:
        return np.zeros((H, W), dtype=bool)

    good = np.zeros_like(comp, dtype=np.uint8)
    areas  = stats[1:, cv2.CC_STAT_AREA]
    hs     = stats[1:, cv2.CC_STAT_HEIGHT]
    keep   = np.where((areas >= MIN_REGION_SIZE) & (hs >= MIN_REGION_HEIGHT))[0] + 1
    for k in keep:
        good[lbls == k] = 1

    full = np.zeros((H, W), dtype=bool)
    full[y0:y1, x0:x1] = good.astype(bool)
    return full

def red_vs_green_score(red_mask, green_mask):
    r = cv2.blur(red_mask.astype(np.float32), (HEAT_BLUR_KSIZE, HEAT_BLUR_KSIZE))
    g = cv2.blur(green_mask.astype(np.float32), (HEAT_BLUR_KSIZE, HEAT_BLUR_KSIZE))
    diff = r - g
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2.0 * amax) + 0.5)
    return np.clip(norm * 255.0, 0, 255.0).astype(np.uint8)

def purple_triangles(score, H):
    top_ex = int(H * EXCLUDE_TOP_FRAC)
    bot_ex = int(H * EXCLUDE_BOTTOM_FRAC)

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    if top_ex: dark[:top_ex, :] = 0
    if bot_ex: dark[-bot_ex:, :] = 0
    dark = cv2.morphologyEx(dark, cv2.MORPH_OPEN, MORPH_SE, iterations=1)

    total_dark = int(dark.sum())
    if total_dark == 0:
        return [], None

    frac_thresh = int(np.ceil(MIN_DARK_FRACTION * total_dark))
    n_lbl, lbls, stats, _ = cv2.connectedComponentsWithStats(dark, 8)
    if n_lbl <= 1:
        return [], None

    tris = []
    for lbl in range(1, n_lbl):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area >= MIN_DARK_RED_AREA and area >= frac_thresh:
            ys, xs = np.where(lbls == lbl)
            if ys.size == 0:
                continue
            y_top = ys.min()
            x_mid = int(xs[ys == y_top].mean())
            tris.append((int(x_mid), int(y_top)))

    if not tris:
        return [], None

    best = min(tris, key=lambda xy: xy[1])
    return tris, best

# ===== Bearing-based Jake triangle selection (kept; no drawing) =====
def signed_degrees_from_vertical(dx, dy):
    if dx == 0 and dy == 0:
        return 0.0
    return -math.degrees(math.atan2(dx, -dy))

def select_triangle_by_bearing(tri_positions, jx, jy, target_deg, min_dy=6):
    best_i, best_deg, best_err = -1, None, None
    for i, (xt, yt) in enumerate(tri_positions):
        dx = xt - jx
        dy = yt - jy
        if dy >= -min_dy:  # must be ahead (above) Jake a little
            continue
        deg = signed_degrees_from_vertical(dx, dy)
        err = abs(deg - target_deg)
        if (best_err is None) or (err < best_err):
            best_i, best_deg, best_err = i, deg, err
    return best_i, best_deg, best_err

# =======================
# Frame post-processing (NO rendering)
# =======================
def process_frame_post(frame_bgr, yolo_res):
    H, W = frame_bgr.shape[:2]
    if yolo_res is None or yolo_res.masks is None:
        return None, 0, 0, 0.0, 0.0, None, None, None, None, [], None, None

    t0_to_cpu = time.perf_counter()

    mdata = yolo_res.masks.data  # torch tensor (n,h,w)
    if mdata is None or mdata.numel() == 0:
        t1 = time.perf_counter()
        return None, 0, 0, (t1 - t0_to_cpu) * 1000.0, 0.0, None, None, None, None, [], None, None

    if getattr(yolo_res, "boxes", None) is None or getattr(yolo_res.boxes, "cls", None) is None:
        t1 = time.perf_counter()
        return None, 0, 0, (t1 - t0_to_cpu) * 1000.0, 0.0, None, None, None, None, [], None, None

    classes_np = yolo_res.boxes.cls.detach().to('cpu', non_blocking=True).numpy().astype(int)
    mask_count = int(mdata.shape[0])

    rail_sel_np = (classes_np == RAIL_ID)
    if not rail_sel_np.any():
        t1 = time.perf_counter()
        return None, 0, mask_count, (t1 - t0_to_cpu) * 1000.0, 0.0, None, None, None, None, [], None, None

    rail_sel = torch.from_numpy(rail_sel_np).to(mdata.device, non_blocking=True)
    union = mdata[rail_sel].any(dim=0).to(dtype=torch.uint8)
    union_cpu = union.cpu().numpy()

    t1_to_cpu = time.perf_counter()
    to_cpu_ms = (t1_to_cpu - t0_to_cpu) * 1000.0

    t0_post = time.perf_counter()

    rail_mask = cv2.resize(union_cpu, (W, H), interpolation=cv2.INTER_NEAREST).astype(bool)
    green = highlight_rails_mask_only_fast(frame_bgr, rail_mask)
    red   = np.logical_and(rail_mask, np.logical_not(green))
    score = red_vs_green_score(red, green)
    tri_positions, tri_best = purple_triangles(score, H)

    # Bearing-based selection (not printed)
    lane_name = lane_name_from_point(JAKE_POINT)
    target_deg = LANE_TARGET_DEG[lane_name]
    jx, jy = JAKE_POINT
    best_idx, best_deg, _ = select_triangle_by_bearing(tri_positions, jx, jy, target_deg, min_dy=6)

    t1_post = time.perf_counter()
    post_ms = (t1_post - t0_post) * 1000.0

    return tri_best, len(tri_positions), mask_count, to_cpu_ms, post_ms, None, None, None, None, tri_positions, best_idx, best_deg

# =======================
# Single-frame execution with prints (NO overlay saving, NO thread pools)
# =======================
def run_pipeline_headless_single():
    paths = (
        glob.glob(str(frames_dir/"frame_*.jpg")) +
        glob.glob(str(frames_dir/"frame_*.png")) +
        glob.glob(str(frames_dir/"*.jpg")) +
        glob.glob(str(frames_dir/"*.png"))
    )
    paths = sorted(set(paths))
    if not paths:
        raise FileNotFoundError(f"No images in: {frames_dir}")
    if SHOW_FIRST_N is not None:
        paths = paths[:SHOW_FIRST_N]

    N = len(paths)

    for idx, p in enumerate(paths, 1):
        img, read_ms = load_image_with_time(p)
        fname = os.path.basename(p)

        if img is None:
            print(f"[{idx}/{N}] {fname}  read {read_ms:.1f} | infer 0.0 | to_cpu 0.0 | post 0.0 | masks 0 | triangles 0 => proc 0.0 ms")
            continue

        # Inference
        t0_inf = time.perf_counter()
        try:
            with torch.no_grad():
                res_list = model.predict(
                    img, task="segment", imgsz=IMG_SIZE, device=device,
                    conf=CONF, iou=IOU, verbose=False, half=half, max_det=MAX_DET
                )
        except Exception as e:
            # Soft fallback if device unstable mid-run
            if str(device) == "mps":
                try:
                    torch.mps.empty_cache()
                except Exception:
                    pass
            print(f"[{idx}/{N}] {fname}  read {read_ms:.1f} | infer FAIL ({e}) | to_cpu 0.0 | post 0.0 | masks 0 | triangles 0 => proc 0.0 ms")
            continue
        t1_inf = time.perf_counter()
        infer_ms = (t1_inf - t0_inf) * 1000.0

        yres = res_list[0] if (isinstance(res_list, (list, tuple)) and len(res_list) > 0) else None

        # Post
        tri_best_xy, tri_count, mask_count, to_cpu_ms, post_ms, *_ = process_frame_post(img, yres)

        proc_ms = infer_ms + to_cpu_ms + post_ms
        print(f"[{idx}/{N}] {fname}  "
              f"read {read_ms:.1f} | infer {infer_ms:.1f} | "
              f"to_cpu {to_cpu_ms:.1f} | post {post_ms:.1f} | "
              f"masks {mask_count} | triangles {tri_count} "
              f"=> proc {proc_ms:.1f} ms")

# =======================
# Entry
# =======================
if __name__ == "__main__":
    # Better crash diagnostics when run as a script (has no effect on hard aborts, but helps)
    try:
        import faulthandler; faulthandler.enable()
    except Exception:
        pass
    run_pipeline_headless_single()


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs
[1/144] frame_00000.png  read 38.0 | infer 76.5 | to_cpu 1.2 | post 96.7 | masks 2 | triangles 1 => proc 174.4 ms
[2/144] frame_00001.png  read 53.1 | infer 84.9 | to_cpu 1.2 | post 79.6 | masks 3 | triangles 1 => proc 165.6 ms
[3/144] frame_00002.png  read 37.1 | infer 56.8 | to_cpu 0.4 | post 103.5 | masks 3 | triangles 1 => proc 160.8 ms
[4/144] frame_00003.png  read 38.5 | infer 54.9 | to_cpu 0.5 | post 113.8 | masks 2 | triangles 1 => proc 169.2 ms
[5/144] frame_00004.png  read 37.3 | infer 54.6 | to_cpu 0.4 | post 158.3 | masks 5 | triangles 1 => proc 213.3 ms
[6/144] frame_00005.png  read 38.4 | infer 60.4 | to_cpu 1.2 | post 162.1 | masks 5 | triangles 1 => proc 223.6 ms
[7/144] frame_00006.png  read 42.2 | infer 51.0 | to_cpu 0.4 | post 164.5 | masks 6 | triangles 2 => proc 215.8 ms
[8/144] frame_00007.png  read 38.2 | infer 54.1 | to_cpu 1.1 | post 166.2 | masks 4 | triangles 2 => proc 221